In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df= pd.read_csv('/content/drive/MyDrive/TFM/Raw_CSV/venues_2023_22_21.csv', sep="	", decimal=".", na_values=['(null)'], low_memory=False)
df.head()

In [ ]:

id_unique=df['unique_venue_id'].unique().tolist()
up=1000
lw=0
len(id_unique[lw:up])
for lw in range(0, len(id_unique), 50000):
  print(lw)
  up= lw+50000
  if up> len(id_unique): sel= id_unique[lw:]
  print(up )
  sel= id_unique[lw:up]
  print(sel,'\n')


In [ ]:
up=1000
lw=0
sel= id_unique[lw:up]


df_filtrado = df[df['unique_venue_id'].isin(sel)]



In [ ]:
from time import time
import datetime

timeIni = time()
df_filtrado= crear_new_latest_live(df_filtrado)
timeEnd = time()
timeDiff = datetime.datetime.fromtimestamp(timeEnd-timeIni).strftime("%M:%S:%f")

print("Tiempo necesario para su cómputo: ", timeDiff)

<ipython-input-21-8463b7b46fb3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aux['new_latest_live_date']=pd.NaT


Tiempo necesario para su cómputo:  05:12:064760


In [ ]:
len(id_unique)

198934

In [ ]:
df['latest_live_date'] = pd.to_datetime(df['latest_live_date'], errors='coerce')
df['disabled_date'] = pd.to_datetime(df['disabled_date'], errors='coerce')
df['first_live_date'] = pd.to_datetime(df['first_live_date'], errors='coerce')
df['date_day'] = pd.to_datetime(df['date_day'])

# Validacion churn

In [ ]:
#Validacion
def calcular_meses_aux(df):
  meses= (df['latest_live_date'].year- df['disabled_date'].year)*12 + (df['latest_live_date'].month- df['disabled_date'].month)
  return meses

df['away'] = df.apply(calcular_meses_aux, axis=1)



In [ ]:
def year_disb (df):
  return df['disabled_date'].year

df['year_disable'] =df.apply(year_disb, axis=1)

In [ ]:
#1800 records en donde pasaron mas de 3 meses entre disable y latest y que tienen churn en algun momento
aa = df.loc[(df['away']>3	) & (df['year_disable']>2021) & (df['churn_mes']==0),['date_day', 'unique_venue_id','first_live_date','latest_live_date' ,'disabled_date', 'away', 'churn_mes' ]]
bb= df.loc[(df['churn_mes']==1),['date_day', 'unique_venue_id','first_live_date','latest_live_date' ,'disabled_date', 'away', 'churn_mes' ]]

result= pd.merge(aa, bb, on="unique_venue_id", how="left")

In [ ]:
#hay 409 casos en donde no tiene nunca churn
result_f= result[result['churn_mes_y'].isnull()]
result_f

,date_day_x,unique_venue_id,first_live_date_x,latest_live_date_x,disabled_date_x,away_x,churn_mes_x,date_day_y,first_live_date_y,latest_live_date_y,disabled_date_y,away_y,churn_mes_y
400,2022-08-01,11000295241,2015-03-27,2022-07-25,2022-03-30,4.0,0,NaT,NaT,NaT,NaT,NaN,NaN
401,2022-09-01,11000295241,2015-03-27,2022-09-01,2022-03-30,6.0,0,NaT,NaT,NaT,NaT,NaN,NaN
402,2022-10-01,11000295241,2015-03-27,2022-09-19,2022-03-30,6.0,0,NaT,NaT,NaT,NaT,NaN,NaN
403,2022-11-01,11000295241,2015-03-27,2022-09-19,2022-03-30,6.0,0,NaT,NaT,NaT,NaT,NaN,NaN
404,2022-12-01,11000295241,2015-03-27,2022-09-19,2022-03-30,6.0,0,NaT,NaT,NaT,NaT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9155,2022-07-01,11000420375,2021-11-30,2022-06-13,2022-01-11,5.0,0,NaT,NaT,NaT,NaT,NaN,NaN
9206,2022-07-01,11000421557,2022-01-14,2022-07-01,2022-02-16,5.0,0,NaT,NaT,NaT,NaT,NaN,NaN
9235,2022-08-01,11000420375,2021-11-30,2022-06-13,2022-01-11,5.0,0,NaT,NaT,NaT,NaT,NaN,NaN
9259,2022-10-01,11000421557,2022-01-14,2022-07-01,2022-02-16,5.0,0,NaT,NaT,NaT,NaT,NaN,NaN


In [ ]:
df_aux= df.loc[df['unique_venue_id']==11000024236,:]
df_aux=pd.concat([df_aux, df.loc[df['unique_venue_id']==11000295241,:] ])
df_aux

,date_day,unique_venue_id,created,venue_status,venue_active_from,original_marketplace_venue_type_name,country_code,tier,saas_product,first_live_date,...,plan_active_from,plan_active_to,prepay,prepaid_until_to,discount,plan_discount_end_date,bill_every,plan_name,plan_fee_eur,is_zero_commission_plan
48719,2022-12-01,11000024236,2010-03-19 17:48:27,LIVE,NaN,Skin Clinic,GB,Tier 1,tw_connect,2010-03-19,...,2022-11-09 14:54:05,2999-12-31 00:00:00,0,NaN,1,2023-01-09,1.0,Entry,33.6139,False
48720,2023-01-01,11000024236,2010-03-19 17:48:27,LIVE,NaN,Skin Clinic,GB,Tier 1,tw_connect,2010-03-19,...,2022-11-09 14:54:05,2999-12-31 00:00:00,0,NaN,1,2023-01-09,1.0,Entry,33.6139,False
48721,2023-02-01,11000024236,2010-03-19 17:48:27,LIVE,NaN,Skin Clinic,GB,Tier 1,tw_connect,2010-03-19,...,2022-11-09 14:54:05,2999-12-31 00:00:00,0,NaN,1,2023-01-09,1.0,Entry,33.6139,False
48722,2023-03-01,11000024236,2010-03-19 17:48:27,LIVE,NaN,Skin Clinic,GB,Tier 1,tw_connect,2010-03-19,...,2022-11-09 14:54:05,2999-12-31 00:00:00,0,NaN,1,2023-01-09,1.0,Entry,33.6139,False
48723,2023-04-01,11000024236,2010-03-19 17:48:27,LIVE,NaN,Skin Clinic,GB,Tier 1,tw_connect,2010-03-19,...,2022-11-09 14:54:05,2999-12-31 00:00:00,0,NaN,1,2023-01-09,1.0,Entry,33.6139,False
48724,2023-05-01,11000024236,2010-03-19 17:48:27,LIVE,NaN,Skin Clinic,GB,Tier 1,tw_connect,2010-03-19,...,2022-11-09 14:54:05,2999-12-31 00:00:00,0,NaN,1,2023-01-09,1.0,Entry,33.6139,False
48725,2023-06-01,11000024236,2010-03-19 17:48:27,LIVE,NaN,Skin Clinic,GB,Tier 1,tw_connect,2010-03-19,...,2022-11-09 14:54:05,2999-12-31 00:00:00,0,NaN,1,2023-01-09,1.0,Entry,33.6139,False
2716087,2022-02-01,11000024236,2010-03-19 17:48:27,NaN,NaN,Skin Clinic,GB,Tier 1,tw_connect,2013-12-30,...,NaN,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN
3102919,2022-10-01,11000024236,2010-03-19 17:48:27,DISABLED,NaN,Skin Clinic,GB,Tier 1,tw_connect,2010-03-19,...,NaN,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN
3163137,2022-01-01,11000024236,2010-03-19 17:48:27,NaN,NaN,Skin Clinic,GB,Tier 1,tw_connect,2013-12-30,...,NaN,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN


In [ ]:
#57 venues con este caso.
result_f['unique_venue_id'].nunique()

57

In [ ]:
aux= df.loc[df['unique_venue_id']==11000024236	 , ['date_day', 'unique_venue_id', 'first_live_date','latest_live_date' ]].sort_values(by=['date_day'] , ascending=True)

In [ ]:
#2022-02-01
fecha_aux='2021-07-01'

upper_latest_live_date = aux.loc[aux['date_day']<fecha_aux , 'latest_live_date'].max()
current_latest_live_date = aux.loc[aux['date_day']==fecha_aux , 'latest_live_date'].max()
new_latest_live_date = max(current_latest_live_date, upper_latest_live_date)
new_latest_live_date

Timestamp('2013-12-30 00:00:00')

In [ ]:
def crear_new_latest_live(df_aux):
  df_aux['new_latest_live_date']=pd.NaT
  #for row in aux.rows:
  for index, row in df_aux.iterrows():#Cambiar el aux por df mas chico
    id_aux= row['unique_venue_id']
    fecha_aux= row['date_day']
    aux= df.loc[df['unique_venue_id']==id_aux	 , ['date_day', 'unique_venue_id', 'first_live_date','latest_live_date' ]]
    upper_latest_live_date = aux.loc[aux['date_day']<fecha_aux , 'latest_live_date'].max()
    #print(upper_latest_live_date)
    current_latest_live_date = aux.loc[aux['date_day']==fecha_aux , 'latest_live_date'].max()
    #print(current_latest_live_date)
    new_latest_live_date = max(current_latest_live_date, upper_latest_live_date)
    df_aux.loc[index, 'new_latest_live_date'] =new_latest_live_date
  return df_aux



In [ ]:
df.to_csv('/content/drive/MyDrive/TFM/Raw_CSV/venues_2023_07_01.csv')

In [ ]:
df_aux.loc[:,['date_day', 'unique_venue_id', 'first_live_date','latest_live_date', 'new_latest_live_date' ]].sort_values(by=[ 'unique_venue_id', 'date_day'] , ascending=True)

In [ ]:
current_latest_live_date

48719   2022-11-09
Name: latest_live_date, dtype: datetime64[ns]

In [ ]:


res= pd.merge(df[['date_day', 'unique_venue_id','latest_live_date']], df[['date_day', 'unique_venue_id','latest_live_date']], on="unique_venue_id", how="inner")
res.head()

In [ ]:
upper_latest_live_date

NaT

In [ ]:
max(current_latest_live_date, upper_latest_live_date)

Timestamp('2022-11-09 00:00:00')

In [ ]:
#Ejemplo 1 : no esperamos que pasen los 3 meses antes de decir que se va.

df.loc[df['unique_venue_id']==11000024236	 , ['date_day', 'unique_venue_id','churn_mes', 'first_live_date','latest_live_date' ,'disabled_date' ,'reactivated_status' , 'venue_status']].sort_values(by=['date_day'] , ascending=True)

,date_day,unique_venue_id,churn_mes,first_live_date,latest_live_date,disabled_date,reactivated_status,venue_status
3590715,2021-07-01,11000024236,0,2013-12-30,2013-12-30,NaT,none,NaN
3644349,2021-08-01,11000024236,0,2013-12-30,2013-12-30,NaT,none,NaN
3590716,2021-09-01,11000024236,0,2013-12-30,2013-12-30,NaT,none,NaN
3436036,2021-10-01,11000024236,0,2013-12-30,2013-12-30,NaT,none,NaN
3436037,2021-11-01,11000024236,0,2013-12-30,2013-12-30,NaT,none,NaN
3799828,2021-12-01,11000024236,0,2013-12-30,2013-12-30,NaT,none,NaN
3163137,2022-01-01,11000024236,0,2013-12-30,2013-12-30,NaT,none,NaN
2716087,2022-02-01,11000024236,1,2013-12-30,2010-03-19,2022-01-31,none,NaN
3382454,2022-03-01,11000024236,1,2010-03-19,2010-03-19,2022-02-02,none,DISABLED
4389397,2022-04-01,11000024236,1,2010-03-19,2010-03-19,2022-02-02,none,DISABLED


In [ ]:
#Ejemplo 2 : la data no se actualiza, y luego se actualiza cuando ya volvio

df.loc[df['unique_venue_id']==11000295241	 , ['date_day', 'unique_venue_id','churn_mes', 'first_live_date','latest_live_date' ,'disabled_date' ,'reactivated_status']].sort_values(by=['date_day'] , ascending=True)

,date_day,unique_venue_id,churn_mes,first_live_date,latest_live_date,disabled_date,reactivated_status
3086659,2021-07-01,11000295241,0,2015-03-26,2015-03-26,NaT,none
3028820,2021-08-01,11000295241,0,2015-03-26,2015-03-26,NaT,none
3086660,2021-09-01,11000295241,0,2015-03-26,2015-03-26,NaT,none
4320207,2021-10-01,11000295241,0,2015-03-26,2015-03-26,NaT,none
4320208,2021-11-01,11000295241,0,2015-03-26,2015-03-26,NaT,none
2854856,2021-12-01,11000295241,0,2015-03-26,2015-03-26,NaT,none
475776,2022-01-01,11000295241,0,2015-03-26,2015-03-26,NaT,none
475777,2022-02-01,11000295241,0,2015-03-26,2021-12-31,NaT,none
475778,2022-03-01,11000295241,0,2015-03-26,2021-12-31,NaT,none
475779,2022-04-01,11000295241,0,2015-03-26,2022-03-29,NaT,none


In [ ]:
#Ejemplo 3 :disable date> latest live date y no tiene varible churn

df.loc[df['unique_venue_id']==11000420375	 , ['date_day', 'unique_venue_id','churn_mes', 'first_live_date','latest_live_date' ,'disabled_date' ,'reactivated_status']].sort_values(by=['date_day'] , ascending=True)

,date_day,unique_venue_id,churn_mes,first_live_date,latest_live_date,disabled_date,reactivated_status
3795559,2021-12-01,11000420375,0,2021-11-29,2021-11-29,NaT,none
3549852,2022-01-01,11000420375,0,2021-11-29,2021-11-29,NaT,none
3099287,2022-02-01,11000420375,0,2021-11-29,2021-11-30,2022-01-09,none
3341868,2022-03-01,11000420375,0,2021-11-29,2021-11-30,2022-01-09,none
4198081,2022-04-01,11000420375,0,2021-11-29,2021-11-30,2022-01-09,none
3448972,2022-05-01,11000420375,0,2021-11-29,2021-11-30,2022-01-09,none
3396260,2022-06-01,11000420375,0,2021-11-29,2021-11-30,2022-01-09,none
3212052,2022-07-01,11000420375,0,2021-11-30,2022-06-13,2022-01-11,reactivated
4141200,2022-08-01,11000420375,0,2021-11-30,2022-06-13,2022-01-11,reactivated
4383963,2022-09-01,11000420375,0,2021-11-30,2022-06-13,2022-01-11,reactivated


##Creacion de variables:
- cantidad de markets por mes de original_marketplace_venue_type_name
- dicotomica saas_product
- Antiguedad ( date_day-latest_live_date (meses)
- reactivada (first <> latest live date)


In [ ]:
df['original_marketplace_venue_type_name'].value_counts()

Hair Salon                   1189965
Beauty Salon                 1027126
Massage & Therapy Centre      199332
Fitness Centre                179595
Nail Salon                    177456
Wellness Centre               142728
Treatment Room - Beauty        89255
Day Spa                        81274
Skin Clinic                    71593
Mobile Beauty                  53981
Home-based Venue               38551
Medical Spa                    34945
Tanning Salon                  34299
Dental Clinic                  31913
Barbershop                     31183
Hotel Spa                      26713
Waxing Salon                   25453
Treatment Room - Wellness      13455
Mobile Massage                 11985
Brow Bar                        8734
Makeup Studio                   8001
Weight Loss Clinic              6170
Yoga Studio                     4608
Pilates Studio                  2640
Destination Spa                 1742
Chiropractic Clinic             1608
Treatment Room - Spa            1593
C

In [ ]:
df.groupby(['unique_venue_id', 'date_day'])['original_marketplace_venue_type_name'].nunique().sort_values(ascending=False)

unique_venue_id  date_day  
11000000002      2021-07-01    1
11000371792      2023-06-01    1
11000371794      2022-04-01    1
                 2022-05-01    1
                 2022-06-01    1
                              ..
12000022921      2022-10-01    0
                 2022-09-01    0
                 2022-08-01    0
                 2022-07-01    0
12000008325      2022-06-01    0
Name: original_marketplace_venue_type_name, Length: 4434772, dtype: int64

In [ ]:
#ejemplo de 3 records con la misma fecha
df.loc[df['unique_venue_id']==12000027472	 , ['date_day', 'unique_venue_id','original_marketplace_venue_type_name', 'created','first_live_date','latest_live_date' ,'disabled_date' ,'reactivated_status']].sort_values(by=['date_day'] , ascending=False)

**SAAS PRODUCT**

In [ ]:
df['saas_product'].value_counts()

tw_connect    3613452
tw_pro         993471
Name: saas_product, dtype: int64

In [ ]:
#un solo producto en cada mes
df.groupby(['unique_venue_id', 'date_day'])['saas_product'].nunique().sort_values(ascending=False)

In [ ]:
#pueden haber cambiado de producto (hay casos que tienen 2)
df.groupby(['unique_venue_id'])['saas_product'].nunique().sort_values(ascending=False)

In [ ]:
#df['tw_connect']=1
#df.loc[df['saas_product']=='tw_pro'	 ,'tw_connect']=0

In [ ]:
#creacion de la variable
df['tw_connect'] = df['saas_product'].apply(lambda x: 0 if x == 'tw_pro' else 1)


**ANTIGUEDAD**

In [ ]:
df['date_day']

In [ ]:
df['date_day']- df['latest_live_date']

In [ ]:
#creacion de la variable
def calcular_meses(df):
  meses= (df['date_day'].year- df['latest_live_date'].year)*12 + (df['date_day'].month- df['latest_live_date'].month)
  return meses

df['antiguedad'] = df.apply(calcular_meses, axis=1)

In [ ]:
df.loc[:, ['unique_venue_id','date_day', 'latest_live_date', 'antiguedad']].sample(20)

,unique_venue_id,date_day,latest_live_date,antiguedad
654298,11000304934,2023-03-01,2015-08-24,91.0
1028693,11000322185,2022-06-01,2016-09-21,69.0
940933,11000312533,2022-02-01,2016-01-26,73.0
193749,11000270452,2022-03-01,2013-11-21,100.0
1756395,11000398228,2022-08-01,2020-02-14,30.0
3918088,11000318807,2021-09-01,2016-06-29,63.0
4106745,11000255325,2022-04-01,2011-08-31,128.0
340351,11000287829,2022-01-01,2014-10-25,87.0
1181008,11000328584,2022-04-01,2017-01-10,63.0
24351,11000007212,2022-04-01,2014-06-02,94.0


**REACTIVADA**

In [ ]:
#creacion de la variable

df['diferencia'] = abs((df['latest_live_date'] - df['first_live_date']).dt.days)
df['reactivada'] = np.where(df['diferencia'] > 30, 1, 0)
df = df.drop(['diferencia'], axis=1)


In [ ]:
#validando
df.loc[df['reactivada']==1, ['unique_venue_id','date_day', 'latest_live_date', 'first_live_date', 'reactivada']].sample(20)
#df.loc[:, ['unique_venue_id','date_day', 'latest_live_date', 'first_live_date', 'reactivada']].sample(20)

,unique_venue_id,date_day,latest_live_date,first_live_date,reactivada
2069070,12000009585,2023-02-01,2022-07-05,2021-06-18,1
1086111,11000317188,2022-04-01,2017-02-08,2016-05-19,1
1440806,11000373776,2022-10-01,2022-03-16,2017-11-30,1
1772468,11000388603,2022-10-01,2022-03-08,2019-08-21,1
1847909,11000402856,2023-03-01,2021-12-06,2020-06-22,1
2045275,12000002495,2023-03-01,2018-11-27,2017-12-01,1
3470642,12000008435,2023-01-01,2021-07-01,2021-03-19,1
3169736,11000037317,2021-10-01,2014-06-02,2010-03-19,1
3013635,11000035383,2023-01-01,2014-06-02,2010-03-19,1
3048500,11000008824,2023-05-01,2014-06-02,2009-09-29,1


In [ ]:
#ejemplo de 3 records con la misma fecha
df.loc[df['unique_venue_id']==12000027472	 , :'latest_live_date'].sort_values(by=['date_day'] , ascending=False)

,date_day,unique_venue_id,created,venue_status,venue_active_from,original_marketplace_venue_type_name,country_code,tier,saas_product,first_live_date,latest_live_date
2464290,2023-06-01,12000027472,2021-09-08 09:58:10,DISABLED,NaN,Wellness Centre,FR,NaN,tw_pro,2021-09-01,2021-09-01
2464289,2023-05-01,12000027472,2021-09-08 09:58:10,DISABLED,NaN,Wellness Centre,FR,NaN,tw_pro,2021-09-01,2021-09-01
2464288,2023-04-01,12000027472,2021-09-08 09:58:10,DISABLED,NaN,Wellness Centre,FR,NaN,tw_pro,2021-09-01,2021-09-01
2464287,2023-03-01,12000027472,2021-09-08 09:58:10,DISABLED,NaN,Wellness Centre,FR,NaN,tw_pro,2021-09-01,2021-09-01
2464286,2023-02-01,12000027472,2021-09-08 09:58:10,DISABLED,NaN,Wellness Centre,FR,NaN,tw_pro,2021-09-01,2021-09-01
2464285,2023-01-01,12000027472,2021-09-08 09:58:10,DISABLED,NaN,Wellness Centre,FR,NaN,tw_pro,2021-09-01,2021-09-01
2464284,2022-12-01,12000027472,2021-09-08 09:58:10,LIVE,NaN,Wellness Centre,FR,NaN,tw_pro,2021-09-01,2021-09-01
2464315,2022-11-01,12000027472,2021-09-08 09:58:10,LIVE,NaN,Wellness Centre,FR,NaN,tw_pro,2021-09-01,2021-09-01
2464316,2022-11-01,12000027472,2021-09-08 09:58:10,LIVE,NaN,Wellness Centre,FR,NaN,tw_pro,2021-09-01,2021-09-01
2464283,2022-11-01,12000027472,2021-09-08 09:58:10,LIVE,NaN,Wellness Centre,FR,NaN,tw_pro,2021-09-01,2021-09-01


In [ ]:
#ejemplo de 3 records con la misma fecha
df.loc[df['unique_venue_id']==12000027472	 , 'latest_live_date':]#.sort_values(by=['date_day'] , ascending=False)

,latest_live_date,disabled_date,reactivated_status,is_twconnect_migrated,is_purchasable,is_widget_enabled,is_heavy_saas,first_heavy_saas_date,active_employees,direct_appointments_l30d,...,prepaid_until_to,discount,plan_discount_end_date,bill_every,plan_name,plan_fee_eur,is_zero_commission_plan,away,mes,churn_mes
2464277,2021-09-01,NaT,none,False,NaN,NaN,False,NaN,1.0,0.0,...,NaN,0,NaN,1.0,Partner Saas,0.0,False,NaN,2021-09,0
2464278,2021-09-01,NaT,none,False,NaN,NaN,False,NaN,1.0,0.0,...,NaN,0,NaN,1.0,Partner Saas,0.0,False,NaN,2021-09,0
2464279,2021-09-01,NaT,none,False,True,True,False,NaN,1.0,0.0,...,NaN,0,NaN,1.0,Partner Saas,0.0,False,NaN,2021-09,0
2464280,2021-09-01,NaT,none,False,True,True,False,NaN,1.0,0.0,...,NaN,0,NaN,1.0,Partner Saas,0.0,False,NaN,2021-09,0
2464281,2021-09-01,NaT,none,False,True,True,False,NaN,NaN,0.0,...,NaN,0,NaN,1.0,Partner Saas,0.0,False,NaN,2021-09,0
2464282,2021-09-01,NaT,none,False,False,True,False,NaN,NaN,0.0,...,NaN,0,NaN,1.0,Partner Saas,0.0,False,NaN,2021-09,0
2464283,2021-09-01,NaT,none,False,False,True,False,NaN,NaN,0.0,...,NaN,0,NaN,1.0,Partner Saas,0.0,False,NaN,2021-09,0
2464284,2021-09-01,2022-12-01,none,False,False,True,False,NaN,NaN,0.0,...,NaN,0,NaN,1.0,Partner Saas,0.0,False,-15.0,2021-09,1
2464285,2021-09-01,2022-12-01,none,False,False,False,False,NaN,NaN,0.0,...,NaN,0,NaN,1.0,Partner Saas,0.0,False,-15.0,2021-09,1
2464286,2021-09-01,2022-12-01,none,False,False,False,False,NaN,NaN,0.0,...,NaN,0,NaN,1.0,Partner Saas,0.0,False,-15.0,2021-09,1


In [ ]:
#ejemplo que cambia en 1 dia

df.loc[df['unique_venue_id']==11000419419	 , ['date_day', 'unique_venue_id','churn_mes', 'first_live_date','latest_live_date' ,'reactivada' ,'reactivated_status']].sort_values(by=['date_day'] , ascending=True)

,date_day,unique_venue_id,churn_mes,first_live_date,latest_live_date,reactivada,reactivated_status
3744674,2021-11-01,11000419419,0,2021-10-25,2021-10-26,0,none
4186382,2021-12-01,11000419419,0,2021-10-25,2021-10-26,0,none
3423496,2022-01-01,11000419419,0,2021-10-25,2021-10-26,0,none
2035115,2022-02-01,11000419419,0,2021-10-25,2021-10-26,0,none
2035116,2022-03-01,11000419419,0,2021-10-25,2021-10-26,0,none
2035117,2022-04-01,11000419419,0,2021-10-25,2021-10-26,0,none
2035118,2022-05-01,11000419419,0,2021-10-25,2021-10-26,0,none
2035119,2022-06-01,11000419419,0,2021-10-25,2021-10-26,0,none
2035120,2022-07-01,11000419419,0,2021-10-26,2021-10-26,0,none
2035121,2022-08-01,11000419419,0,2021-10-26,2021-10-26,0,none


# CHURN


In [ ]:
import pandas as pd
import numpy as np

df_venues = df.copy()

# Reemplazamos los valores NaN por 0 en disabled_date
df_venues['disabled_date'].fillna(0, inplace=True)

# Transformamos la variable "disabled_date" a formato fecha
df_venues['disabled_date'] = pd.to_datetime(df_venues['disabled_date'], errors='coerce')

# Si latest_live_date es nulo, seleccionamos first_live_date
df_venues['latest_live_date'].fillna(df_venues['first_live_date'], inplace=True)

# Eliminar todos los registros que tienen valores NaN en latest_live_date
df_venues.dropna(subset=['latest_live_date'], inplace=True)

# Transformamos la variable "latest_live_date" a formato fecha
df_venues['latest_live_date'] = pd.to_datetime(df_venues['latest_live_date'], errors='coerce')

# Creamos una nueva columna 'mes' a partir de la columna 'latest_live_date'
df_venues['mes'] = df_venues['latest_live_date'].dt.to_period('M')

# Calculamos el churn por cada mes
df_venues['date_day'] = pd.to_datetime(df_venues['date_day'])
tres_meses = df_venues['disabled_date'] + pd.DateOffset(months=3)
df_venues['churn'] = np.where((df_venues['disabled_date'] > df_venues['latest_live_date']) & (df_venues['date_day'] > tres_meses), 1, 0)


In [ ]:
df_venues['churn']

0          0
1          0
2          0
3          0
4          0
          ..
4606918    0
4606919    1
4606920    1
4606921    0
4606922    0
Name: churn, Length: 3565062, dtype: int64

#Antiguedad negativa

In [ ]:
aa= df_venues_cami.loc[df_venues_cami['antiguedad']<0,['date_day', 'unique_venue_id', 'antiguedad']]
aa.head()

,date_day,unique_venue_id,antiguedad
2080946,2023-02-01,12000006274,-1.0
2082193,2023-02-01,12000000084,-3.0
2082194,2023-03-01,12000000084,-2.0
2082195,2023-04-01,12000000084,-1.0
2126475,2023-02-01,12000005348,-2.0


In [ ]:
#df.loc[df['unique_venue_id']==12000006274, ['date_day', 'unique_venue_id', 'latest_live_date', 'new_latest_live_date']].sort_values(['date_day'], ascending = False)
#df.loc[df['unique_venue_id']==12000000084, ['date_day', 'unique_venue_id', 'latest_live_date', 'new_latest_live_date']].sort_values(['date_day'], ascending = False)
#df.loc[df['unique_venue_id']==12000018115, ['date_day', 'unique_venue_id', 'latest_live_date', 'new_latest_live_date']].sort_values(['date_day'], ascending = False)

bb= df.loc[:, ['date_day', 'unique_venue_id', 'latest_live_date', 'new_latest_live_date']].drop_duplicates(subset=['unique_venue_id', 'date_day'], keep='first')
bb.head()


,date_day,unique_venue_id,latest_live_date,new_latest_live_date
0,2021-07-01,11000000016,2009-07-20,2009-07-20
1,2021-08-01,11000000016,2009-07-20,2009-07-20
2,2021-09-01,11000000016,2009-07-20,2009-07-20
3,2021-10-01,11000000016,2009-07-20,2009-07-20
4,2021-11-01,11000000016,2009-07-20,2009-07-20


In [ ]:

mergeaabb = pd.merge(aa, bb,  how='left', left_on=['unique_venue_id','date_day'], right_on = ['unique_venue_id','date_day'])
mergeaabb.head()

,date_day,unique_venue_id,antiguedad,latest_live_date,new_latest_live_date
0,2023-02-01,12000006274,-1.0,NaN,NaN
1,2023-02-01,12000000084,-3.0,NaN,NaN
2,2023-03-01,12000000084,-2.0,NaN,NaN
3,2023-04-01,12000000084,-1.0,NaN,NaN
4,2023-02-01,12000005348,-2.0,NaN,NaN


In [ ]:
mergeaabb['new_latest_live_date'].value_counts()

Series([], Name: new_latest_live_date, dtype: int64)